In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import itertools

In [2]:
with open("/home/peter/Downloads/AdventOfCode/day19Input.txt") as fh:
    lines = fh.readlines()
#with open("/home/peter/Downloads/AdventOfCode/day19Test.txt") as fh:
#    lines = fh.readlines()


In [3]:
scanners = {}
scanNum = -1
totalBeacons = 0
allBeacons = []
for line in lines:
    if "scanner" in line:
        scanNum = int(line.split()[2])
    elif "," in line:
        if scanNum not in scanners:
            scanners[scanNum] = []
        nums = [int(x) for x in line.strip().split(",")]
        totalBeacons += len(nums)
        allBeacons.append(nums)
        scanners[scanNum].append(nums)

In [4]:
class scanner:
    def __init__(self,number,inputPoints):
        self.id = number
        self.orientation = -1
        self.points = inputPoints
        self.xzyShuffle = { "x":"x","y":"y","z":"z" }
        self.xzyFlip    = { "x":1,"y":1,"z":1 }
    
    def _getPoint(self,index):
        pass

In [5]:
def manhatDist(p1,p2):
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1]) + abs(p1[2]-p2[2])

pointIDs = {}
def makePointIDs():
    
    for s in scanners:
        #print("doing ",s)
        if s not in pointIDs:
            pointIDs[s] = {}
        i = 0
        for p1 in scanners[s]:
            #print(p1)
            if i not in pointIDs[s]:
                pointIDs[s][i] = {}
            pointIDs[s][i]["xyz"] = p1
            pointIDs[s][i]["dist"] = set()
            for p2 in scanners[s]:
                d = manhatDist(p1,p2)
                pointIDs[s][i]["dist"].add(d)
            #pointDat["dist"] = sorted(pointDat["dist"])
            #pointIDs[s][i] = pointDat
            i += 1
makePointIDs()

In [11]:
matches = []
added = set()
# "3:2" style keys
points = {}
def addSingle(first):
    found = False
    if first in added:
        found = True
    #for group in matches:
        #if first in group:
            #found = True
    if found == False:
        added.add(first)
        matches.append([first])
def addMatch(first,second):
    found = False
    for group in matches:
        if first in group:
            if second not in group:
                group.append(second)
                added.add(second)
            found = True
    if found == False:
        added.add(first)
        added.add(second)
        newGroup = [first,second]
        matches.append(newGroup)
def findMatches():        
    for scan1 in pointIDs:
        if type(scan1) != int:
            continue
        for scan2 in pointIDs:
            if type(scan2) != int:
                continue
            if scan1 == scan2:
                continue
            # match if enough points are similiar
            for p1s in pointIDs[scan1]:
                if type(p1s) != int:
                    continue
                points[f"{scan1}:{p1s}"] = pointIDs[scan1][p1s]
                foundMatch = False
                for p2s in pointIDs[scan2]:
                    if type(p2s) != int:
                        continue
                    s1 = pointIDs[scan1][p1s]["dist"]
                    s2 = pointIDs[scan2][p2s]["dist"]
                    inter = s1.intersection(s2)
                    if len(inter) > 10: #(len(pointIDs[scan1][p1s]["dist"])/2): # why? IDK seems like a good number
                        #print(f"match on {scan1}:{p1s} + {scan2}:{p2s}")
                        addMatch(f"{scan1}:{p1s}",f"{scan2}:{p2s}")
                if foundMatch == False:
                    addSingle(f"{scan1}:{p1s}")
findMatches()
print(len(matches))

351


In [12]:
#print(pointIDs)

In [13]:
def adjust(point, d):
    assert 0<=d<48
    # last bit of d says whether to flip x
    # second to last bit of d says whether to flip y
    # third to last bit of d says whether to flip z
    # rest of d (d//8) identifies which of the 6 permutations

    ret = [point[0], point[1], point[2]]
    for i,p in enumerate(list(itertools.permutations([0, 1, 2]))):
        if d//8 == i:
            ret = [ret[p[0]], ret[p[1]], ret[p[2]]]

    if d%2==1:
        ret[0] *= -1
    if (d//2)%2==1:
        ret[1] *= -1
    if (d//4)%2==1:
        ret[2] *= -1
    return ret

In [14]:
scannerMatches = {}
def makeScannerMatches():
    for m in matches:
        if len(m) > 1:
            #print(m)
            #scanners = []
            for d1 in m:
                s1,p1 = d1.split(":")
                s1 = int(s1)
                p1 = int(p1)

                for d2 in m:
                    if d1 == d2:
                        continue
                    s2,p2 = d2.split(":")
                    s2 = int(s2)
                    p2 = int(p2)
                    if "matches" not in pointIDs[s1][p1]:
                        pointIDs[s1][p1]["matches"] = set()
                    if "scanMatchCount" not in pointIDs[s1]:
                        pointIDs[s1]["scanMatchCount"] = {}
                    if s2 not in pointIDs[s1]["scanMatchCount"]:
                        pointIDs[s1]["scanMatchCount"][s2] = 0
                    pointIDs[s1]["scanMatchCount"][s2] += 1
                    pointIDs[s1][p1]["matches"].add(d2)

                #print(sorted(points[d]["dist"]))
                #print(points[d]["xyz"])
            for i in pointIDs:
                if i not in scannerMatches:
                    scannerMatches[i] = {}
                for j in scanners:
                    if j not in scannerMatches[i]:
                        scannerMatches[i][j] = set()
                    scannerMatches[i][j].update(m)
makeScannerMatches()
        
# total number of beacons is less than 351
solvedScanners = set()
solvedScanners.add(0)
pointIDs[0]["d"] = 0
pointIDs[0]["xyzAdj"] = (0,0,0)

In [15]:
def getTruePoint(solvedScanners,pointIDs,name):
    s1,p1 = name.split(":")
    s1 = int(s1)
    p1 = int(p1)
    if s1 in solvedScanners:
        newPoint = adjust(pointIDs[s1][p1]["xyz"],pointIDs[s1]["d"])
        newPoint = [x+y for x,y in zip(newPoint,pointIDs[s1]["xyzAdj"])]
        return newPoint
    else:
        return None

def runOneMatchScanners(solvedScanners,pointIDs):
    newSet = set()
    #for scan1 in solvedScanners:
    for scan1 in [0]:
        for scan2 in pointIDs[scan1]["scanMatchCount"]:
            if scan2 not in solvedScanners and pointIDs[scan1]["scanMatchCount"][scan2] >= 12:
                print("trying scanner",scan2," against scanner ",scan1)
                adjustVar = 0
                matched = False
                while matched == False:
                    #print("match loop with adjust = ",adjustVar)
                    matched = True
                    # for all points
                    firstPoint = True
                    for p1 in pointIDs[scan1]:
                        if matched == False:
                            break
                        #print("working on point ",p1)
                        if type(p1) != int:
                            continue
                        # find matches for that scanner:
                        if "matches" not in pointIDs[scan1][p1]:
                            continue
                        for match in pointIDs[scan1][p1]["matches"]:
                            #print("working on point ",p1," loop ",match)
                            s2,p2 = match.split(":")
                            s2 = int(s2)
                            p2 = int(p2)
                            #print(scan2," ",s2)
                            if scan2 == s2:
                                #print("testing point ",match)
                                # try match
                                if firstPoint:
                                    firstPoint = False
                                    pointIDs[scan2]["d"] = adjustVar
                                    newPoint = adjust(pointIDs[s2][p2]["xyz"],pointIDs[s2]["d"])
                                    pointIDs[scan2]["xyzAdj"] = [x-y for x,y in zip(getTruePoint(solvedScanners,pointIDs,f"{scan1}:{p1}"),newPoint)]
                                    #print("set xyzAdj: ",newPoint," with ",pointIDs[scan1][p1]["xyz"])
                                    #print("xyzAdj = ",pointIDs[scan2]["xyzAdj"])
                                else:
                                    newPoint = adjust(pointIDs[s2][p2]["xyz"],pointIDs[s2]["d"])
                                    #print("post shuffle", newPoint)
                                    newPoint = [x+y for x,y in zip(newPoint,pointIDs[s2]["xyzAdj"])]
                                    #print(pointIDs[s2][p2]["xyz"]," and ",pointIDs[scan1][p1]["xyz"])
                                    #print("comparing", newPoint," and ",pointIDs[scan1][p1]["xyz"])
                                    if newPoint != getTruePoint(solvedScanners,pointIDs,f"{scan1}:{p1}"):
                                        #print("bad match on ",match)
                                        matched = False
                                        adjustVar += 1
                                        break

                    if matched == True:
                        print("found match with adjust = ",adjustVar)
                        newSet.add(scan2)
                        newSet.update(solvedScanners)
    if len(newSet) > len(solvedScanners):
        solvedScanners = newSet
    return solvedScanners
for i in range(20):
    solvedScanners = runOneMatchScanners(solvedScanners,pointIDs)
    truePoints = set()
    for s1 in solvedScanners:
        if type(s1) == int:
            print("for ",s1)
            for p1 in pointIDs[s1]:
                if type(p1) == int:
                    newPoint = getTruePoint(solvedScanners,pointIDs,f"{s1}:{p1}")
                    point = tuple(newPoint)
                    truePoints.add(point)
    points = {}
    i = 0
    for point in truePoints:
        if point not in scanners[0]:
            scanners[0].append(point)
    #scanners[0] = newList
    makePointIDs()
    matches = []
    findMatches()
    makeScannerMatches()
    pointIDs[0]["d"] = 0
    pointIDs[0]["xyzAdj"] = (0,0,0)
    print("added stuff to pointIDs[0]")

trying scanner 7  against scanner  0
found match with adjust =  29
trying scanner 16  against scanner  0
found match with adjust =  9
trying scanner 2  against scanner  0
found match with adjust =  24
trying scanner 22  against scanner  0
found match with adjust =  44
trying scanner 5  against scanner  0
found match with adjust =  35
trying scanner 1  against scanner  0
found match with adjust =  27
for  0
for  16
for  2
for  1
for  5
for  22
for  7
added stuff to pointIDs[0]
trying scanner 11  against scanner  0
found match with adjust =  10
trying scanner 17  against scanner  0
found match with adjust =  37
trying scanner 19  against scanner  0
found match with adjust =  23
trying scanner 13  against scanner  0
found match with adjust =  5
trying scanner 15  against scanner  0
found match with adjust =  15
trying scanner 3  against scanner  0
found match with adjust =  17
trying scanner 12  against scanner  0
found match with adjust =  47
trying scanner 23  against scanner  0
found m

In [16]:
print(pointIDs[1].keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 'scanMatchCount', 'd', 'xyzAdj'])


In [17]:
print(pointIDs[0][0].keys())


dict_keys(['xyz', 'dist', 'matches'])


In [18]:
print(pointIDs.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24])


In [20]:
#print(scanners)

In [21]:
truePoints = set()
for s1 in solvedScanners:
    if type(s1) == int:
        print("for ",s1)
        for p1 in pointIDs[s1]:
            if type(p1) == int:
                newPoint = getTruePoint(solvedScanners,pointIDs,f"{s1}:{p1}")
                point = tuple(newPoint)
                truePoints.add(point)

for  0
for  1
for  2
for  3
for  4
for  5
for  6
for  7
for  8
for  9
for  10
for  11
for  12
for  13
for  14
for  15
for  16
for  17
for  18
for  19
for  20
for  21
for  22
for  23
for  24


In [22]:
foo = [2, 4,6]
print(pointIDs[0][1])

{'xyz': [309, -819, -775], 'dist': {0, 6153, 8250, 6206, 6210, 6236, 6246, 10345, 4204, 4209, 4211, 6267, 4220, 4221, 6278, 4230, 10377, 4237, 6287, 4242, 6295, 4247, 4249, 6299, 4254, 4265, 6316, 4270, 4280, 2233, 4285, 4293, 6341, 201, 2250, 4300, 4301, 2270, 2286, 2295, 4353, 6414, 4367, 2321, 4373, 278, 2330, 6432, 4393, 4412, 4416, 6465, 8514, 2378, 6480, 4433, 2387, 6488, 2398, 6507, 4461, 4462, 4476, 6535, 2444, 4496, 2450, 4501, 2456, 2460, 6557, 6562, 2478, 6582, 2494, 2498, 2499, 8643, 4550, 6607, 6608, 8668, 8672, 4579, 6634, 8683, 2553, 6669, 4630, 4637, 4667, 6715, 4672, 4697, 6753, 8803, 4717, 2672, 4736, 4745, 4754, 6807, 4767, 4772, 2729, 6850, 2760, 2761, 8905, 4813, 4819, 4835, 4837, 2795, 4848, 4853, 8962, 2823, 4873, 2832, 4882, 4891, 2854, 2855, 9006, 2866, 6964, 4919, 9022, 4939, 2892, 2897, 2899, 7015, 4972, 2940, 4995, 5009, 5024, 7073, 2993, 5042, 5041, 5047, 3003, 3006, 5062, 7131, 3042, 3057, 7155, 5129, 3083, 5135, 1057, 5154, 7201, 7205, 3126, 3147, 5202, 3

In [26]:
print(len(truePoints))
print(len(matches))
#print(pointIDs[1]["xyzAdj"])
#print(pointIDs[1][0]["xyz"])
print(getTruePoint(solvedScanners,pointIDs,f"1:0"))

315
337
[472, 488, 1837]


In [24]:
blah = 0
for point in sorted(truePoints):
    print(point)
    blah += 1
    pass
print(blah)
for match in matches:
    print(match)

(-3303, 1919, 1547)
(-3279, 863, 369)
(-3269, -551, 1585)
(-3237, -782, 614)
(-3231, -708, 636)
(-3230, -817, 731)
(-3222, 1921, 1468)
(-3199, 815, 399)
(-3194, 879, 456)
(-3180, 1970, 1590)
(-3175, -411, 1543)
(-3162, -416, 1629)
(-3046, -4168, 2866)
(-3032, -4077, 3003)
(-3024, -1992, 1526)
(-3015, -1820, 1482)
(-3002, -4099, 2870)
(-2970, -1796, 1504)
(-2954, -3016, 2015)
(-2936, -2935, 2870)
(-2936, 1996, 709)
(-2914, 1887, 751)
(-2907, 2055, 664)
(-2869, -1943, 2699)
(-2857, -3000, 2900)
(-2851, -3046, 1861)
(-2849, -2007, 2764)
(-2828, -4012, 1857)
(-2826, -2871, 2841)
(-2808, -1829, 2817)
(-2790, -3996, 1854)
(-2768, -4015, 1635)
(-2738, 755, 1980)
(-2737, 799, 2006)
(-2732, -3046, 2008)
(-2732, 519, 2001)
(-2517, -78, 1226)
(-2511, 1203, 1098)
(-2496, 76, 1062)
(-2476, -2304, 2427)
(-2445, -3590, 2368)
(-2440, -2416, 2235)
(-2346, 37, 1181)
(-2332, -2444, 2405)
(-2107, 373, 1632)
(-2103, 3065, 720)
(-2045, 325, 1544)
(-1993, -1708, 1895)
(-1992, -334, -1703)
(-1987, -306, -1713

In [25]:
print(solvedScanners)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24}


In [33]:
bigDist = 0
for i in pointIDs:
    for j in pointIDs:
        bigDist = max(manhatDist(pointIDs[i]["xyzAdj"],pointIDs[j]["xyzAdj"]),bigDist)
        
        
print(bigDist)

13192
